In [47]:
# QR-decomposition using Gram-Schmidt Process

A = [[1,0,1],[0,1,1],[1,2,0]]
n = len(A)
a1 = []
a2 = []
a3 = []
for i in range(n):
    a1.append(A[i][0])
    a2.append(A[i][1])
    a3.append(A[i][2])

# u1
u1 = a1


# calculating <a2, u1>
dp21 = 0
for i in range(len(u1)):
    val = a2[i] * u1[i]
    dp21 += val

# calculating norm of u1
norm_u1 = 0
for i in range(len(u1)):
    norm_u1 += u1[i]**2
norm_u1 = norm_u1**(0.5)

# u2
u2 = []
for i in range(len(a2)):
    val = a2[i] - (dp21/norm_u1**2)*u1[i]
    u2.append(val)


# calculating <a3, u1> and <a3, u2>
dp31 = 0
for i in range(len(u1)):
    val = a3[i] * u1[i]
    dp31 += val
dp32 = 0
for i in range(len(u2)):
    val = a3[i] * u2[i]
    dp32 += val

# calculating norm of u2
norm_u2 = 0
for i in range(len(u2)):
    norm_u2 += u2[i]**2
norm_u2 = norm_u2**(0.5)

# u3
u3 = []
for i in range(len(a3)):
    val = a3[i] - ((dp31/norm_u1**2)*u1[i] + (dp32/norm_u2**2)*u2[i])
    u3.append(val)
    
# calculating norm of u2
norm_u3 = 0
for i in range(len(u3)):
    norm_u3 += u3[i]**2
norm_u3 = norm_u3**(0.5)

# orthogonal vector: v1, v2, v3
v1 = []
v2 = []
v3 = []
for i in range(len(u1)):
    v1.append(u1[i] / norm_u1)
    v2.append(u2[i] / norm_u2)
    v3.append(u3[i] / norm_u3)

# matrix Q
Q = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(len(Q)):
    Q[i][0] = v1[i]
    Q[i][1] = v2[i]
    Q[i][2] = v3[i]

# matrix R
R = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(len(R)):
    R[0][0] += a1[i]*v1[i]
    R[0][1] += a2[i]*v1[i]
    R[0][2] += a3[i]*v1[i]
    R[1][1] += a2[i]*v2[i]
    R[1][2] += a3[i]*v2[i]
    R[2][2] += a3[i]*v3[i]

In [61]:
def transpose(A):
    n = len(A)
    m = len(A[0])
    res = []
    for j in range(m):
        row = []
        for i in range(n):
            val = A[i][j]
            row.append(val)
        res.append(row)
    return res

def norm(a):
    n = len(a)
    res = 0
    for i in range(n):
        res += a[i]**2
    res = res**(0.5)
    return res

def inner_product(a,b):
    n = len(a) # n=len(a)=len(b)
    res = 0
    for i in range(n):
        res += a[i]*b[i]
    return res

def normalize(a):
    n = len(a)
    res = []
    for i in range(n):
        val = a[i] / norm(a)
        res.append(val)
    return res

In [122]:
# QR-decomposition using Gram-Schmidt Process

def qr_gram(A):
    At = transpose(A)
    n = len(At)
    norm_list = []
    U = []
    V = []

    for i in range(0,n):
        if i == 0:
            u = At[i]
            U.append(u)
            norm_u = norm(u)
            norm_list.append(norm_u)

        else:
            a = At[i]
            dp_list = []
            for j in range(0,i):
                dp = inner_product(a,U[j])
                dp_list.append(dp)

            u = []
            for j in range(0,n):
                val = a[j]
                for k in range(0, i):
                    val -= (dp_list[k] / norm_list[k]**2)*U[k][j]
                u.append(val)
            norm_u = norm(u)
            norm_list.append(norm_u)
            U.append(u)

        v = normalize(u)
        V.append(v)
    Q = transpose(V)

    R = []
    for i in range(0,n):
        r = []
        for j in range(0,n):
            if i > j:
                val = 0
                r.append(val)
            else:
                val = inner_product(At[j], V[i])
                r.append(val)
        R.append(r)
        
    return Q, R

In [179]:
def outer_product(a, b):
    n1 = len(a)
    n2 = len(b)
    res = []
    for i in range(n1):
        row = []
        for j in range(n2):
            val = a[i] * b[j]
            row.append(val)
        res.append(row)
    return res

def inner_product(a, b):
    n1 = len(a) # n1 = len(b)
    val = 0
    for i in range(n1):
        val += a[i] * b[i]
    return val

def identity(n):
    I = []
    for i in range(n):
        row = []
        for j in range(n):
            if i != j:
                val = 0
                row.append(val)
            else:
                val = 1
                row.append(val)
        I.append(row)
    return I

def subtract(A, B):
    res = []
    for i in range(len(A)):
        row = []
        for j in range(len(B)):
            val = A[i][j] - B[i][j]
            row.append(val)
        res.append(row)
    return res

def householder(v):
    n = len(v)
    outer_mat = outer_product(v,v) #list
    inner_val = inner_product(v,v) #int
    V = []
    for i in range(n):
        row = []
        for j in range(n):
            val = (outer_mat[i][j] / inner_val) * 2
            row.append(val)
        V.append(row)
    I = identity(n)
    H = subtract(I, V)
    return H

def matmul(A, B):
    m = len(A)
    r = len(A[0]) # r = len(B)
    n = len(B[0])
    res = []
    for i in range(m):
        row = []
        for j in range(n):
            val = 0
            for k in range(r):
                val += A[i][k] * B[k][j]
            row.append(val)
        res.append(row)
    return res

def sign(a1):
    if a1[0] >= 0:
        sign = 1
    else:
        sign = -1
    return sign

In [192]:
# QR-decomposition using Householder matrix

A = [[1,-1,4],[1,4,-2],[1,4,2],[1,-1,0]]

# calculating A1, H1
A1 = A

a1 = transpose(A1)[0]

e1 = [1]
for i in range(1, len(a1)):
    e1.append(0)

v1 = []
for i in range(len(a1)):
    v1.append(a1[i] + sign(a1)*norm(a1)*e1[i])

H1 = householder(v1)

# calculating A2, H2
AH1 = matmul(H1,A1)

A2 = []
for i in range(1, len(AH1)):
    val = AH1[i][:0]+AH1[i][1:]
    A2.append(val)

a2 = transpose(A2)[0]

e2 = [1]
for i in range(1, len(a2)):
    e2.append(0)

v2 = []
for i in range(len(a2)):
    v2.append(a2[i] + sign(a2)*norm(a2)*e2[i])

H2 = householder(v2)

# calculating A3, H3
AH2 = matmul(H2,A2)

A3 = []
for i in range(1, len(AH2)):
    val = AH2[i][:0]+AH2[i][1:]
    A3.append(val)

a3 = transpose(A3)[0]

e3 = [1]
for i in range(1, len(a3)):
    e3.append(0)

v3 = []
for i in range(len(a3)):
    v3.append(a3[i] + sign(a3)*norm(a3)*e3[i])

H3 = householder(v3)

# converting H2, H3
c_H2 = identity(len(A))
for i in range(len(H2)):
    for j in range(len(H2)):
        c_H2[i+1][j+1] = H2[i][j]

c_H3 = identity(len(A))
for i in range(len(H3)):
    for j in range(len(H3)):
        c_H3[i+2][j+2] = H3[i][j]

# Q = H1 * H2 * H3
Q = matmul(matmul(H1,c_H2),c_H3)

# R = H3 * H2 * H1 * A
R = matmul(matmul(matmul(c_H3,c_H2),H1),A)